**DEBUG CODE**


In [ ]:
from importlib import reload

In [ ]:
#this also takes care of all the imports
import analysis_helper
import analysis_graphs
from analysis_helper import *
from analysis_graphs import *
reload(analysis_helper)
reload(analysis_graphs)

**DEBUG CODE END**

# Block construction agents

This is the main analysis file for the [block construction task](https://github.com/cogtoolslab/block_construction). 

The data should be loaded in by a dataframe produced by experiment_runner.py

In [ ]:
#this also takes care of all the imports
from analysis_helper import *
from analysis_graphs import *

Let's load the results of the experiment

In [ ]:
df_paths = ['../beam_search.pkl']

In [ ]:
df_paths = ['../search_test.pkl']

In [ ]:
#load all experiments as one dataframe
df = pd.concat([pd.read_pickle(l) for l in df_paths])

In [ ]:
display(df)

We'll have data for the following agents and on the following silhouettes:

In [ ]:
smart_short_agent_names(df['agent'].unique())

In [ ]:
[short_world_name(w) for w in df['world'].unique()]

In [ ]:
#TODO plot nice view of worlds used

If we only want to look at results for a particular silhouette or agent, the following syntax can be used:

In [ ]:
#df = df[df['world'].str.contains('int_struct_15')]

## Overview over agents
All agents use pure F1 score to judge the value of intermediate states.

| Agent |  | Parameter |
|:--|:--|:--|
| Random (special case of BFS) | Randomly chooses a legal action. | *None* |
| Breadth first search | Agent performs breadth first search on the tree of all possible actions and chooses the sequence of actions that has the highest average reward over the next *planning depth* steps | Horizon: how many steps in advance is the tree of possible actions searched? |
| MCTS | Implements Monte Carlo Tree Search | Horizon: the number of rollouts per run |
| Naive Q learning | Implements naive Q learning with an epsilon-greedy exploration policy | Maximum number of episodes: how many episodes per run |
| A* search | Implements A* search algorithm. Runs until winning state is found or an upper limit of steps is reached. | *None* |
| Beam search | Implements beam search: searches tree of possible action, but only keeps the best *beam size* actions at each iteration | Beam size: the number of states kept under consideration at every step |

### Glossary

**Run**: training (if applicable) and running one agent on one particular silhouette.

**Silhouette**: the particular outline (and set of baseblocks) that the agent has to reconstruct.

**State**: state of the blockworld environment consisting of the blocks that have already been placed in it.

## Success

### Rate of perfect reconstruction per agent

How often does an agent succeed in getting a perfect reconstruction?

In [ ]:
mean_win_per_agent(df)

### F1 score
What [F1 score](https://en.wikipedia.org/wiki/F1_score) does the agent achieve? Since F1 score decreases if an agent keeps building after being unable to perfectly recreate the structure, we look at the peak of F1 score for every run. 

So here is the average peak F1 score per agent conditioned on outcome of the run.

In [ ]:
mean_peak_score_per_agent(df)

## Failure kinds
In run where the agent fails to achieve perfect reconstruction, what is the reason for the failure?

**Full** indicates that no further block can be placed.
**Unstable** indicates the structure has collapsed.
**Did not finish** means that the agent hasn't finished building either because it terminated or it reached the limit on number of steps (40).

In [ ]:
mean_failure_reason_per_agent(df)

## Greediness
Do agents prefer a greedy policy (using large blocks to cover much area) or a conservative policy of using smaller blocks and more steps?

### Number of steps taken
On average, how many steps does an agent take before the end of the run? 

Looking at the average number of steps for runs with perfect reconstruction ("win") tells us whether an agent builds with larger or smaller blocks. 

Looking at the average number of steps for runs with failed reconstructions ("fail") tells whether the failures occur early or late in the process. Since many failures are due to the agent simply filling everything with blocks this number is likely high and not very informative.

In [ ]:
avg_steps_to_end_per_agent(df)

### Average F1 score during run
On average, what is the average F1 score taken over every action up to the peak of F1 score for a particular run? For runs conditioned on perfect reconstructions, what is the growth rate of F1?

The higher this number is, the more F1 score is gained early on in the run (ie. a logaritmic looking curve of F1 score). 
Note that the bars conditioned on winning runs all have a peak F1 score of 1 and are thus directly comparable.

In [ ]:
mean_avg_area_under_curve_to_peakF1_per_agent(df)

### Average F1 score over time
What is the average F1 score over time? 

Decreasing line indicates the behavior of the agent to keep choosing the least-worst action if a perfect reconstruction is no longer possible.

For runs that terminate early, the last F1 score is kept as to not show outliers in the later part of the graph. Thus, a perfect reconstruction at step 8 is counted as a score of 1 for the last 12 steps. 

In [ ]:
graph_mean_F1_over_time_per_agent(df)

### Average block size over time
What is the average size of the block placed at a certain step?

Note that only runs that aren't finished at a given step are included in the calculation of the mean/std, so later steps might be less informative.

In [ ]:
graph_avg_blocksize_over_time_per_agent(df)

## Consistency
How consistent are different runs of the same agent on the same silhouette?

## Locality bias
Does the agent prefer to place a block on top of or next to the last placed block?